In [1]:
import numpy as np
import os
import pandas as pd
import shutil
import math
%load_ext rpy2.ipython


/home/infertility_bmi/anaconda3/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


In [3]:
# Parse phenotype file and update outcome-info variable list and data-coding-ordinal-info data coding files
outputDirName = "Test"
outputDirPath = "../../Results/" + outputDirName
dataCoding = pd.read_csv('../../data/data-coding-ordinal-info.txt')
variableInfo = pd.read_csv('../../data/outcome-info.tsv',delimiter="\t")
phenotypeInfo = pd.read_csv('../../data/simulated_phenotypes.csv')
phenotypeCoding = pd.read_csv('../../data/Codings.csv')

# Remove not needed variables from variable info
phenotypeVariables = list(phenotypeInfo.columns)
phenotypeVariables.pop(0)
for index, variableID in enumerate(phenotypeVariables):
    phenotypeVariables[index] = int(variableID.split('x')[1].split('_')[0])
variableInfo = variableInfo.loc[variableInfo['FieldID'].isin(phenotypeVariables)]

# Change text phenotype information to proper coding value
codingsDict = {}
for col in phenotypeInfo.columns:
    print(col)
    if not col[0] == "x":
        continue
    print(int(col.split('x')[1].split('_')[0]))
    codingNumberIndex = variableInfo.index[(variableInfo['FieldID'] == int(col.split('x')[1].split('_')[0])).tolist()].tolist()[0]
    codingNumber = variableInfo['DATA_CODING'].loc[codingNumberIndex]
    if math.isnan(codingNumber):
        continue
    if codingsDict.get(codingNumber) is None:
        phenotypeCodeIndices = phenotypeCoding.index[(phenotypeCoding['Coding'] == codingNumber).tolist()].tolist()
        newCoding = {}
        for index in phenotypeCodeIndices:
            newCoding[phenotypeCoding['Meaning'].loc[index]] = phenotypeCoding['Value'].loc[index]
        codingsDict[codingNumber] = newCoding
    coding = codingsDict[codingNumber]
    phenotypeInfo[col].replace(to_replace=coding, inplace=True)
#     for value in phenotypeInfo[col].iteritems():
#         if isinstance(value[1], str):
#             phenotypeInfo[col].loc[int(value[0])] = coding[value[1]]
    
if os.path.exists(outputDirPath) and os.path.isdir(outputDirPath):
    shutil.rmtree(outputDirPath)
os.mkdir(outputDirPath)
os.mkdir(outputDirPath + "/input")

variableInfo.to_csv(outputDirPath + "/input/outcome-info.tsv", index=False, sep="\t")
dataCoding.to_csv(outputDirPath + "/input/data-coding.csv", index=False)
phenotypeInfo.to_csv('../../data/simulated_phenotypes_encoded.csv', index=False)

/home/infertility_bmi/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (29,30,169,177,184,186,192,193,194,195,196,197,198,201,202,203,204,205,206,207,218,219,220,221,222,223,224,225,272,273,274,275,276,277,278,279,307,310,311,314) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/infertility_bmi/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3,4,5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


userId
x31_0_0
31
x34_0_0
34
x52_0_0
52
x84_0_0
84
x84_0_1
84
x84_0_2
84
x84_0_3
84
x84_0_4
84
x84_0_5
84
x84_1_0
84
x84_1_1
84
x84_1_2
84
x84_1_3
84
x84_1_4
84
x84_1_5
84
x84_2_0
84
x84_2_1
84
x84_2_2
84
x84_2_3
84
x84_2_4
84
x84_2_5
84
x84_3_0
84
x84_3_1
84
x84_3_2
84
x84_3_3
84
x84_3_4
84
x84_3_5
84
x189_0_0
189
x190_0_0
190
x191_0_0
191
x2090_0_0
2090
x2090_1_0
2090
x2090_2_0
2090
x2090_3_0
2090
x2178_0_0
2178
x2178_1_0
2178
x2178_2_0
2178
x2178_3_0
2178
x2405_0_0
2405
x2405_1_0
2405
x2405_2_0
2405
x2405_3_0
2405
x2443_0_0
2443
x2443_1_0
2443
x2443_2_0
2443
x2443_3_0
2443
x2453_0_0
2453
x2453_1_0
2453
x2453_2_0
2453
x2453_3_0
2453
x2966_0_0
2966
x2966_1_0
2966
x2966_2_0
2966
x2966_3_0
2966
x2976_0_0
2976
x2976_1_0
2976
x2976_2_0
2976
x2976_3_0
2976
x3894_0_0
3894
x3894_1_0
3894
x3894_2_0
3894
x3894_3_0
3894
x20001_0_0
20001
x20001_0_1
20001
x20001_0_2
20001
x20001_0_3
20001
x20001_0_4
20001
x20001_0_5
20001
x20001_1_0
20001
x20001_1_1
20001
x20001_1_2
20001
x20001_1_3
20001
x20001_

In [8]:
# Run phe-WAS through PHESANT
!Rscript phenomeScan.r \
--phenofile="../../data/simulated_phenotypes_encoded.csv" \
--traitofinterestfile="../../data/rs2032631_G.csv" \
--variablelistfile="../../Results/Test/input/outcome-info.tsv" \
--datacodingfile="../../Results/Test/input/data-coding.csv" \
--traitofinterest="exposure" \
--resDir="../../Results/rs2032631_G/" --standardise="FALSE" --userId="userId"

[1] "Running with all traits in phenotype file: ../../data/simulated_phenotypes_encoded.csv"
[1] "Validating phenotype data ..."
[1] "Number of columns in phenotype file: 316"
[1] "Phenotype file validated"
[1] "Validating trait of interest data ..."
[1] "Number of columns in trait of interest file:2"
[1] "Trait of interest file validated"
[1] "Loading phenotypes ..."
  [1] "userId"      "x31_0_0"     "x34_0_0"     "x52_0_0"     "x84_0_0"    
  [6] "x84_0_1"     "x84_0_2"     "x84_0_3"     "x84_0_4"     "x84_0_5"    
 [11] "x84_1_0"     "x84_1_1"     "x84_1_2"     "x84_1_3"     "x84_1_4"    
 [16] "x84_1_5"     "x84_2_0"     "x84_2_1"     "x84_2_2"     "x84_2_3"    
 [21] "x84_2_4"     "x84_2_5"     "x84_3_0"     "x84_3_1"     "x84_3_2"    
 [26] "x84_3_3"     "x84_3_4"     "x84_3_5"     "x189_0_0"    "x190_0_0"   
 [31] "x191_0_0"    "x2090_0_0"   "x2090_1_0"   "x2090_2_0"   "x2090_3_0"  
 [36] "x2178_0_0"   "x2178_1_0"   "x2178_2_0"   "x2178_3_0"   "x2405_0_0"  
 [41] "x2405_1_0"   "

Loading required package: nnet
Loading required package: lmtest
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Waiting for profiling to be done...
Loading required package: MASS
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting

Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be done...
Waiting for profiling to be 

In [2]:
!Rscript phenomeScan.r --test

[1] "Running with all traits in phenotype file: ../testWAS/data/phenotypes.csv"
[1] "Validating phenotype data ..."
[1] "Number of columns in phenotype file: 38"
[1] "Phenotype file validated"
[1] "Validating trait of interest data ..."
[1] "Number of columns in trait of interest file:2"
[1] "Trait of interest file validated"
[1] "Loading phenotypes ..."
[1] "Loading trait of interest file ..."
[1] "Loading confounders from phenotypes file ..."
[1] "Adjusting for age, sex and genotype chip"
[1] "Number of rows in confounder data: 1000"
[1] "Number of INCOMPLETE rows removed from confounder data: 0"
[1] "Number of rows in confounder data: 1000"
[1] "Confounder columns:"
[1] "userID"     "x21022_0_0" "x31_0_0"    "genoChip"  
[1] "Phenotype file has 1000 rows with 1000 not NA for trait of interest (exposure)."
[1] "Phenotype and trait of interest data files merged, with 1000 examples"
[1] "Loading indicator fields from phenotypes file ..."
[1] "Loading required related variable(s):"
[1] 